# MLP 
RSAT=0.01 and 90 spectra per class

In [1]:
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import MaxNLocator
import numpy as np
from numpy import asarray
import pandas as pd
import math
import seaborn as sns  #heat map
import glob # batch processing of images


import matplotlib.font_manager as fm
import random
import sys
import os

from sklearn.datasets import make_regression
# import tensorflow as tf
from sklearn.metrics import confusion_matrix    #confusion matrix
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# Collect all the font names available to matplotlib
font_names = [f.name for f in fm.fontManager.ttflist]
# print(font_names)

from scipy import signal
from scipy import interpolate

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve 
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score


from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF

#Sklearn model saving and loading
from joblib import dump, load

if '../../' not in sys.path:
    sys.path.append('../../')

from aimos.spectral_datasets.THz_datasets import THz_data

from aimos.misc.utils import simple_plotter


#Set random seed
os.environ['PYTHONHASHSEED'] = str(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
# tf.random.set_seed(42)  
# tf.random.get_global_generator().reset_from_seed(42)
np.random.seed(42)
random.seed(42)

from oneida import THz_mixture_data
from oneida_utils import concentrations_to_one_hot_encode, create_mixture_names
from oneida_utils import simple_spectrum_fig, simple_plot_raw_scores, plot_spectrum_with_scores, multiclass_roc_auc_score, multiclass_sensitivity_specificity_score, multiclass_sensitivity_threshold_score
from oneida_scoring_tools import calc_AMCAS, is_cui_present, is_cui_present_in_mult
from aimos.misc.utils import classifier_internals
from aimos.misc.utils import clf_post_processor
from oneida_utils import mixture_names_to_one_hot_encode
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from oneida_grad_cam import grad_cam

from stats import stats
stats(n_compounds=8)

2022-10-30 12:36:52.199576: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Total 1-C combinations: 8
Total 2-C combinations: 28
Total 3-C combinations: 56
Total 4-C combinations: 70
Total 5-C combinations: 56
Total 6-C combinations: 28
Total 7-C combinations: 8
Total 8-C combinations: 1
Total combinations: 255


# Retrieve training(D)+ validation(V) mixtures and its variables

In [2]:
# initialize
TAAT = 0.001 
ASAT=0.01
RSAT=0.01

m = THz_mixture_data(resolution=0.016, pressure='1 Torr', verbosity=False)
m.initiate_THz_mixture_data(TAAT = TAAT, 
                               ASAT=ASAT, 
                               RSAT=RSAT)

reduced_labels = m.labels
reduced_labels.remove('')
reduced_labels.remove(' ')
reduced_labels.remove('Diluent')
print('reduced_labels', reduced_labels)


# data_filename = "datasets/TSMCN-5-L-229_DV_04-09-2022_time_22-26-37.pkl"
data_filename = "datasets/TSMCN-8-L-229_DV__TAAT_0.001_ASAT_0.01_RSAT_0.01_20-10-2022_time_23-16-29_class_cnt_90.pkl"
DV = pd.read_pickle(data_filename)
y = DV['y'].to_numpy()
mixture_names = DV['mixture_names'].to_numpy()
y_concentrations = DV[['y_c0', 'y_c1', 'y_c2','y_c3', 'y_c4', 'y_c5', 'y_c6', 'y_c7']].to_numpy()
X = DV.drop(['y','mixture_names', 'y_c0', 'y_c1', 'y_c2','y_c3', 'y_c4', 'y_c5', 'y_c6', 'y_c7'],axis=1).to_numpy()

final_neuron_number = np.unique(y, axis=0).shape[0]
print('Number of neurons in the final layer :', final_neuron_number)

print('labels from class:', m.labels)

Components :  [[0 1 2 3 4 5 6 7 8]]
Components shape :  (1, 9)
TAAT =  0.001
ASAT =  0.01
RSAT =  0.01
reduced_labels ['$C_2H_5OH$', '$CH_3CHO$', '$CH_3Cl$', '$CH_3CN$', '$CH_3OH$', '$H_2CO$', '$HCOOH$', '$HNO_3$']
Number of neurons in the final layer : 255
labels from class: ['$C_2H_5OH$', '$CH_3CHO$', '$CH_3Cl$', '$CH_3CN$', '$CH_3OH$', '$H_2CO$', '$HCOOH$', '$HNO_3$']


# preview one test mixture spectra using simple plotter

In [3]:
# idx = 239
# simple_plotter(m.frequencies,X[idx],linewidth=0.5,color='black',label=mixture_names[idx], 
#                    majorsize=6,minorsize=2,width=1, labelsize=8,legendsize=3, legendloc=2,  
#                    labelpad=4,fontsize='medium',fontweight='bold',
#                   xmajormplloc=0.5,xminormplloc=0.2, tickdirection='out')

# print(y_concentrations[idx])
# print(reduced_labels)

In [4]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(mixture_names)

mixture_types=le.classes_
print(mixture_types)

['$CH_3CHO$' '$CH_3CHO$+$CH_3CN$' '$CH_3CHO$+$CH_3CN$+$CH_3OH$'
 '$CH_3CHO$+$CH_3CN$+$CH_3OH$+$HCOOH$'
 '$CH_3CHO$+$CH_3CN$+$CH_3OH$+$HCOOH$+$HNO_3$'
 '$CH_3CHO$+$CH_3CN$+$CH_3OH$+$HNO_3$'
 '$CH_3CHO$+$CH_3CN$+$CH_3OH$+$H_2CO$'
 '$CH_3CHO$+$CH_3CN$+$CH_3OH$+$H_2CO$+$HCOOH$'
 '$CH_3CHO$+$CH_3CN$+$CH_3OH$+$H_2CO$+$HCOOH$+$HNO_3$'
 '$CH_3CHO$+$CH_3CN$+$CH_3OH$+$H_2CO$+$HNO_3$'
 '$CH_3CHO$+$CH_3CN$+$HCOOH$' '$CH_3CHO$+$CH_3CN$+$HCOOH$+$HNO_3$'
 '$CH_3CHO$+$CH_3CN$+$HNO_3$' '$CH_3CHO$+$CH_3CN$+$H_2CO$'
 '$CH_3CHO$+$CH_3CN$+$H_2CO$+$HCOOH$'
 '$CH_3CHO$+$CH_3CN$+$H_2CO$+$HCOOH$+$HNO_3$'
 '$CH_3CHO$+$CH_3CN$+$H_2CO$+$HNO_3$' '$CH_3CHO$+$CH_3Cl$'
 '$CH_3CHO$+$CH_3Cl$+$CH_3CN$' '$CH_3CHO$+$CH_3Cl$+$CH_3CN$+$CH_3OH$'
 '$CH_3CHO$+$CH_3Cl$+$CH_3CN$+$CH_3OH$+$HCOOH$'
 '$CH_3CHO$+$CH_3Cl$+$CH_3CN$+$CH_3OH$+$HCOOH$+$HNO_3$'
 '$CH_3CHO$+$CH_3Cl$+$CH_3CN$+$CH_3OH$+$HNO_3$'
 '$CH_3CHO$+$CH_3Cl$+$CH_3CN$+$CH_3OH$+$H_2CO$'
 '$CH_3CHO$+$CH_3Cl$+$CH_3CN$+$CH_3OH$+$H_2CO$+$HCOOH$'
 '$CH_3CHO$+$CH_3Cl$+$CH_3CN

In [9]:
#split intro train and validation set

#seeds used 123,237, 786
from sklearn.model_selection import train_test_split

global_indices=range(0, X.shape[0])
print(global_indices)

# (np.expand_dims(X,-1)
TRAIN_SIZE=0.60
VAL_SIZE=1-TRAIN_SIZE

x_train, x_val, y_train, y_val, train_indices, val_indices = train_test_split(X, y, global_indices, train_size=TRAIN_SIZE,
                                                   test_size=VAL_SIZE,
                                                   random_state=786,
                                                    stratify=y

                                                   )

print('X_train shape:', x_train.shape)
print('y_ohe_train shape:', y_train.shape)

print('X_val shape:', x_val.shape)
print('y_ohe_val shape:', y_val.shape)


print("All:", np.bincount(y) / float(len(y))*100  )
print("Training:", np.bincount(y_train) / float(len(y_train))*100  )
print("Validation:", np.bincount(y_val) / float(len(y_val))*100  )


range(0, 22950)
X_train shape: (13770, 229)
y_ohe_train shape: (13770,)
X_val shape: (9180, 229)
y_ohe_val shape: (9180,)
All: [0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686 0.39215686
 0

In [10]:
from oneida_model import get_callbacks, get_optimizer, compile_and_fit, TSMCN_12_L_229

In [11]:
import sklearn.neural_network 
from sklearn.neural_network import MLPClassifier


In [17]:
classifier_MLP = MLPClassifier(solver='lbfgs', activation='relu', alpha=1e-7, learning_rate='constant', max_iter=2000,  
                           hidden_layer_sizes=(300, 600), random_state=0).fit(x_train, y_train)

y_pred = classifier_MLP.predict(x_val)


---------------------------- MLP -------------------------------
Fraction Correct[Accuracy]:
0.9604575163398693
Samples Correctly Classified:
(array([   0,    1,    2, ..., 9177, 9178, 9179]),)
Samples Incorrectly Classified:
(array([   4,   65,   79,   96,   97,  110,  134,  149,  177,  217,  230,
        298,  323,  326,  362,  418,  425,  466,  469,  485,  496,  568,
        640,  643,  729,  749,  772,  781,  834,  845,  855,  857,  858,
        867,  918,  934,  938,  948,  954,  993, 1011, 1018, 1037, 1044,
       1058, 1101, 1137, 1141, 1152, 1176, 1178, 1212, 1228, 1268, 1275,
       1290, 1296, 1320, 1324, 1344, 1349, 1403, 1407, 1426, 1464, 1515,
       1631, 1634, 1651, 1662, 1664, 1669, 1670, 1671, 1675, 1708, 1736,
       1765, 1769, 1791, 1804, 1829, 1885, 1886, 1889, 1897, 1920, 1967,
       2008, 2093, 2096, 2100, 2124, 2165, 2183, 2195, 2206, 2210, 2214,
       2219, 2264, 2291, 2368, 2373, 2412, 2448, 2459, 2486, 2496, 2497,
       2541, 2588, 2607, 2618, 2619, 2661, 

In [18]:
classifier_internals(y_pred, y_val, y_train, 'MLP')

---------------------------- MLP -------------------------------
Fraction Correct[Accuracy]:
0.9604575163398693
Samples Correctly Classified:
(array([   0,    1,    2, ..., 9177, 9178, 9179]),)
Samples Incorrectly Classified:
(array([   4,   65,   79,   96,   97,  110,  134,  149,  177,  217,  230,
        298,  323,  326,  362,  418,  425,  466,  469,  485,  496,  568,
        640,  643,  729,  749,  772,  781,  834,  845,  855,  857,  858,
        867,  918,  934,  938,  948,  954,  993, 1011, 1018, 1037, 1044,
       1058, 1101, 1137, 1141, 1152, 1176, 1178, 1212, 1228, 1268, 1275,
       1290, 1296, 1320, 1324, 1344, 1349, 1403, 1407, 1426, 1464, 1515,
       1631, 1634, 1651, 1662, 1664, 1669, 1670, 1671, 1675, 1708, 1736,
       1765, 1769, 1791, 1804, 1829, 1885, 1886, 1889, 1897, 1920, 1967,
       2008, 2093, 2096, 2100, 2124, 2165, 2183, 2195, 2206, 2210, 2214,
       2219, 2264, 2291, 2368, 2373, 2412, 2448, 2459, 2486, 2496, 2497,
       2541, 2588, 2607, 2618, 2619, 2661, 

In [8]:
# tf.keras.utils.plot_model(model, to_file="RESULTS/TSMCN_8_L_229.png", show_shapes=True, rankdir='TB', dpi=150)

In [29]:
# #Sklearn model saving and loading
# from joblib import dump, load
# import datetime
# from datetime import date, datetime
# now = datetime.now()
# dt_string = now.strftime("%d-%m-%Y_time_%H-%M-%S")
# model.save('model/' + model_name + '_' + dt_string + '.hdf5')
# np.save('model/' + model_name + '_' + dt_string + 'history' + '.npy',history.history)
# np.save('model/' + model_name + '_' + dt_string + 'epoch' + '.npy',history.epoch)


In [20]:
import tensorflow as tf

In [21]:
y_ohe = concentrations_to_one_hot_encode(y_concentrations).astype('int64')
y_train_ohe = y_ohe[train_indices]
y_val_ohe = y_ohe[val_indices]
y_train_ohe_tensor = tf.convert_to_tensor(y_train_ohe, np.int64)
y_val_ohe_tensor = tf.convert_to_tensor(y_val_ohe, np.int64)

2022-10-30 13:09:20.025277: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-10-30 13:09:20.057251: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: Quadro RTX 4000 computeCapability: 7.5
coreClock: 1.545GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 387.49GiB/s
2022-10-30 13:09:20.057314: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-10-30 13:09:20.062409: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-10-30 13:09:20.062524: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-10-30 13:09:20.063722: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.10

In [22]:
# plt.figure(dpi=150)
# plt.scatter(history.epoch,history.history['accuracy'], color = 'red', label = 'training')
# plt.scatter(history.epoch,history.history['val_accuracy'], color = 'blue', label = 'validation')
# plt.legend(loc=4)
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.savefig(r'RESULTS/results_figures/' + model_name + '_accuracies.png', bbox_inches='tight')

# plt.figure(dpi=150)
# plt.scatter(history.epoch,history.history['SparseCatCrossentropy'], color = 'red', label = 'training')
# plt.scatter(history.epoch,history.history['val_SparseCatCrossentropy'],color = 'blue', label = 'validation')
# plt.legend(loc=1)
# plt.xlabel('Epoch')
# plt.ylabel('Sparse categorical crossentropy loss')
# plt.savefig(r'RESULTS/results_figures/'+ model_name + '_sparse_cat_losses.png', bbox_inches='tight')

In [23]:
# predict experimental mixtures

In [31]:


exps = ['2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Mix 50% Dilute CM-ACN.xlsx',
'2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Pure Mix CM-ACN.xlsx',
'2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Mix 90% Dilute CM-ACN.xlsx',
'3 Comp-mix_ 90+% CH3OH + 5-% CH3CN + 5-% CH3CL/0.9 CH3OH + 0.05 CH3CN + 0.05 CH3Cl - 1.xlsx',
'3 Comp-mix_ 90+% CH3OH + 5-% CH3CN + 5-% CH3CL/0.9 CH3OH + 0.05 CH3CN + 0.05 CH3Cl - 2.xlsx',
'4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2.xlsx',
'4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/90% Dilute in N2 - 0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN.xlsx'       ]

true_label=[81,81,81,82,82,19,19]

exp_path = '../../data/Mixture_exp_data/'
exp_filepath = '4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2.xlsx'


def classify_exp(exp_path,exp_filepath,mixture_types,true_label, excel=True):
    all_unique_labels= mixture_types
    if excel:
        df_exp1 = pd.read_excel(exp_path + exp_filepath)
    else:
        df_exp1 = pd.read_csv(exp_path + exp_filepath)


    freq_exp1 = df_exp1[df_exp1.columns[0]].to_numpy()
    abs_exp1 = df_exp1[df_exp1.columns[1]].to_numpy()
    
#     fft_filter(freq_exp1, abs_exp1, factor=75)

    dfy_resampled= signal.resample(abs_exp1, len(m.frequencies))
    dfx_resampled= signal.resample(freq_exp1, len(m.frequencies))
#     import pdb; pdb.set_trace()
    pred_exp_label = classifier_MLP.predict(np.reshape(dfy_resampled, (1,len(m.frequencies)) ))
    
    

    
#     print(score_predictions)
    print('Experiment name: ',exp_filepath.split('/')[0])
    print('File name: ',exp_filepath.split('/')[1])
    print('predicted index ', pred_exp_label)
    print('predicted label', mixture_types[pred_exp_label])
#     display(sorted_scoresDf)
    print('-----------------------------------------------------')
    



#     import pdb; pdb.set_trace()
    return pred_exp_label
idx = 0 
for experiment in exps:
    pred_exp_label = classify_exp(exp_path,experiment,mixture_types, true_label[idx],excel=True)
    idx+=1
    

Experiment name:  2 Comp-mix_ 30 % CH3Cl - 70% CH3CN
File name:  Mix 50% Dilute CM-ACN.xlsx
predicted index  [216]
predicted label ['$C_2H_5OH$+$CH_3Cl$+$CH_3CN$+$H_2CO$+$HCOOH$+$HNO_3$']
-----------------------------------------------------
Experiment name:  2 Comp-mix_ 30 % CH3Cl - 70% CH3CN
File name:  Pure Mix CM-ACN.xlsx
predicted index  [209]
predicted label ['$C_2H_5OH$+$CH_3Cl$+$CH_3CN$+$CH_3OH$+$H_2CO$+$HCOOH$+$HNO_3$']
-----------------------------------------------------
Experiment name:  2 Comp-mix_ 30 % CH3Cl - 70% CH3CN
File name:  Mix 90% Dilute CM-ACN.xlsx
predicted index  [202]
predicted label ['$C_2H_5OH$+$CH_3Cl$+$CH_3CN$']
-----------------------------------------------------
Experiment name:  3 Comp-mix_ 90+% CH3OH + 5-% CH3CN + 5-% CH3CL
File name:  0.9 CH3OH + 0.05 CH3CN + 0.05 CH3Cl - 1.xlsx
predicted index  [202]
predicted label ['$C_2H_5OH$+$CH_3Cl$+$CH_3CN$']
-----------------------------------------------------
Experiment name:  3 Comp-mix_ 90+% CH3OH + 5-% 

# classify pure compounds

In [33]:
exps = ['1 comp/v2Paper Ethanol 1 Torr.csv',
       '1 comp/Methanol 1 Torr WDL 8.csv',
       '1 comp/Formic Acid 1 Torr WDL 5 Notch Shift.csv',
       '1 comp/Acetaldehyde 1 Torr WDL 6 Notch.csv',
       '1 comp/Acetaldehyde 0p5 Torr WDL 5.csv',
       '1 comp/1-Sweep v2 CH3OH 220-330-1 Torr.csv',
       '1 comp/1-Sweep CH3Cl 220-330-1 Torr.csv',
       '1 comp/1-Sweep CH3Cl 220-330-0p5 Torr.csv',
       '1 comp/1-Sweep 0p5 Torr-CH3CN.csv',]
true_label=[120,112,248,0,0,80,80,64]

exp_path = '../../data/Mixture_exp_data/'
exp_filepath = '4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2.xlsx'


def classify_exp(exp_path,exp_filepath,mixture_types,true_label, excel=True):
    all_unique_labels= mixture_types
    if excel:
        df_exp1 = pd.read_excel(exp_path + exp_filepath)
    else:
        df_exp1 = pd.read_csv(exp_path + exp_filepath)


    freq_exp1 = df_exp1[df_exp1.columns[0]].to_numpy()
    abs_exp1 = df_exp1[df_exp1.columns[1]].to_numpy()
    
#     fft_filter(freq_exp1, abs_exp1, factor=75)

    dfy_resampled= signal.resample(abs_exp1, len(m.frequencies))
    dfx_resampled= signal.resample(freq_exp1, len(m.frequencies))
#     import pdb; pdb.set_trace()
    pred_exp_label = classifier_MLP.predict(np.reshape(dfy_resampled, (1,len(m.frequencies)) ))
    
    

    
#     print(score_predictions)
    print('Experiment name: ',exp_filepath.split('/')[0])
    print('File name: ',exp_filepath.split('/')[1])
    print('predicted index ', pred_exp_label)
    print('predicted label', mixture_types[pred_exp_label])
#     display(sorted_scoresDf)
    print('-----------------------------------------------------')
    



#     import pdb; pdb.set_trace()
    return pred_exp_label
idx = 0 
for experiment in exps:
    pred_exp_label = classify_exp(exp_path,experiment,mixture_types, true_label[idx],excel=False)
    idx+=1


Experiment name:  1 comp
File name:  v2Paper Ethanol 1 Torr.csv
predicted index  [146]
predicted label ['$C_2H_5OH$+$CH_3CHO$+$CH_3Cl$+$CH_3CN$+$CH_3OH$+$H_2CO$+$HCOOH$+$HNO_3$']
-----------------------------------------------------
Experiment name:  1 comp
File name:  Methanol 1 Torr WDL 8.csv
predicted index  [234]
predicted label ['$C_2H_5OH$+$CH_3OH$+$HCOOH$']
-----------------------------------------------------
Experiment name:  1 comp
File name:  Formic Acid 1 Torr WDL 5 Notch Shift.csv
predicted index  [178]
predicted label ['$C_2H_5OH$+$CH_3CHO$+$HCOOH$']
-----------------------------------------------------
Experiment name:  1 comp
File name:  Acetaldehyde 1 Torr WDL 6 Notch.csv
predicted index  [146]
predicted label ['$C_2H_5OH$+$CH_3CHO$+$CH_3Cl$+$CH_3CN$+$CH_3OH$+$H_2CO$+$HCOOH$+$HNO_3$']
-----------------------------------------------------
Experiment name:  1 comp
File name:  Acetaldehyde 0p5 Torr WDL 5.csv
predicted index  [146]
predicted label ['$C_2H_5OH$+$CH_3CHO$+$C

IndexError: list index out of range